# TODO:
- Use proper HTML library to clean up clunky html strings
- Allow resolution change to saved images to save space.
- Pass language to Whisper???
- Change from Whisper to Facebook's fairseq MMC? https://github.com/facebookresearch/fairseq/tree/main/examples/mms

In [41]:
import cv2
import os
from collections import Counter
from tqdm import tqdm
import whisper
import torch
import moviepy.editor as mp
import subprocess
from datetime import timedelta
from yattag import Doc

ADD_FURIGANA = True
WIDTH = 200
src_dir = 'Input'
out_dir = 'Output'
files = [os.path.join(src_dir, f) for f in os.listdir(src_dir)]

if ADD_FURIGANA:
    from furigana.furigana import split_furigana

In [42]:
def file_to_line_list(filename, encoding='utf-8-sig'):
        line_list = []
        with open(filename, 'r', encoding=encoding) as file:
            for line in file:
                line_list += [line.replace('\n', '')]
        return line_list

def chunk_sub_idx_to_list(sub_line_list):
        """
            Pass in a list where each line is a line in the subtitle file
            Example:
            ['1', '00:00:00,000 --> 00:00:04,430', 'おはようございます', '2', ...]

            return a list where each list item is another list where each item is specific to its index
            Example:
            [['1', '00:00:00,000 --> 00:00:04,430', 'おはようございます'], ['2', ...], ...]
        """
        lines_indexed = []
        tmp = []
        for i, line in enumerate(sub_line_list):
            if line == '':
                continue

            tmp += [line]
            if len(tmp) > 3:
                digit, timestamp = tmp[-2:]
                if digit.strip().isdigit() and '-->' in timestamp:
                    lines_indexed += [tmp[:-2]]
                    tmp = tmp[-2:]
        return lines_indexed
    
def srt_time_to_seconds(time_line):
    def timestr_to_sec(time_str):
        h, m, s_str = time_str.split(':')
        s, ms = s_str.split(',')
        return int(h)*60*60 + int(m)*60 + int(s) + int(ms)/1000 

    start_time_str, stop_time_str = time_line.split(' --> ')
    start_time = timestr_to_sec(start_time_str)
    stop_time = timestr_to_sec(stop_time_str)

    return start_time, stop_time

In [43]:
def add_image(image, w=200, h=200):
    basename = os.path.splitext(os.path.basename(image))[0]
    return f'<img src="{image}" alt="{basename}" width="{w}" height="{h}" class="center">'

def add_audio_clip(audio_file):
    return \
    f"""<audio controls class="center">
    <source src="{os.path.basename(audio_file)}" type="audio/mpeg">
    Your browser does not support the audio element.
    </audio> 
    """

def add_sub(sub):
    return f'<div>{sub}</div>'    

def create_html(name, content):
    return \
    f"""<!DOCTYPE html>
    <html>
    <style>
    div {{text-align: center;}}
    .center {{
      display: block;
      margin-left: auto;
      margin-right: auto;
    }}
    </style>
    <title>{name}</title>
    <head>
    </head>
    <body>
    <h1>{name}</h1>
    {content}
    </body>
    </html>
    """

In [44]:
def add_furigana(text):

    w_furigana = ''
    for pair in split_furigana(text):
        if len(pair)==2:
            kanji,hira = pair
            w_furigana +=  f"<ruby><rb>{kanji}</rb><rt>{hira}</rt></ruby>"
        else:
            w_furigana += pair[0]
    return w_furigana

In [45]:
def create_html_list(prj_name, vid_file, lines_indexed, audio=True, pad=0.25):

    video_capture = cv2.VideoCapture(vid_file)
    
    if audio:
        mp_video = mp.VideoFileClip(vid_file)

    html_list = []

    for r in lines_indexed:
        line_idx, time_str = r[:2]
        sub_list = r[2:]
        
        if ADD_FURIGANA:
            tmp = []
            for s in sub_list:
                try:
                    tmp += [add_furigana(s)]
                except:
                    tmp += [s]
                
            sub_list = tmp

        start, stop = srt_time_to_seconds(time_str)
        time_ms = int(1000*((stop - start)/2 + start))

        video_capture.set(cv2.CAP_PROP_POS_MSEC, time_ms)
        success, image = video_capture.read()

        if success:
            new_filename = prj_name + '_' + str(time_ms) + '.jpg'
            path = os.path.join(prj_dir, new_filename)
            if not(os.path.exists(path)):
                cv2.imwrite(path, image)
                
            if audio:
                audio_filename = os.path.splitext(path)[0] + '.mp3'
                mp_video.subclip(max(0, start - pad), stop+pad).audio.write_audiofile(audio_filename)

            h, w = image.shape[:-1]
            ratio = h/w

            html_list += [add_image(new_filename, w=WIDTH, h=int(WIDTH*ratio))]
            html_list += [add_sub(s) for s in sub_list]
            
            if audio:
                html_list += [add_audio_clip(audio_filename)]
            
    return html_list

In [46]:
device = whisper.torch.device('cuda' if whisper.torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [47]:
# Create an mp3 from video file
def extract_audio(file, savename='', track_number=0): 
    def extract_ffmpeg(input_file, output_file, audio_track):
        command = f'ffmpeg -i "{input_file}" -map 0:a:{audio_track} "{output_file}"'
        result = subprocess.run(command, shell=True, text=True, capture_output=True)
        print("stdout:", result.stdout)
        print("stderr:", result.stderr)

    noext = os.path.splitext(file)[0]
    savename = noext + '.mp3' if savename == '' else savename

    # Default to using MoviePy
    if track_number == 0:
        v = mp.VideoFileClip(f)
        v.audio.write_audiofile(savename)
        
    # Extract different track, need ffmpeg
    else:
        try:
            extract_ffmpeg(file, savename, track_number)
        except:
            print('Cant process ffmpeg. Ensure you have ffmpeg installed.')

In [48]:
def transcribe_audio(transcription, savepath):
    segments = transcription['segments']

    for segment in segments:
        startTime = str(0)+str(timedelta(seconds=int(segment['start'])))+',000'
        endTime = str(0)+str(timedelta(seconds=int(segment['end'])))+',000'
        text = segment['text']
        if len(text) == 0:
            continue
        
        segmentId = segment['id']+1
        segment = f"{segmentId}\n{startTime} --> {endTime}\n{text[1:] if text[0] is ' ' else text}\n\n"

        with open(savepath, 'a', encoding='utf-8') as srtFile:
            srtFile.write(segment)

In [49]:
TRACK_NUMBER = 2

for f in files:
    noext = os.path.splitext(f)[0]
    
    subtitle_file = noext + '.srt'
    
    # Use SRT doesn't exists create it
    if not(os.path.exists(subtitle_file)):
        
        audio_file = noext + '.mp3'

        # Check if audio exists
        if not(os.path.exists(audio_file)):
            print(f'Extracting mp3 from {f}...')
            extract_audio(f, savename=audio_file, track_number=TRACK_NUMBER)
            print('Finished extracting audio.')
            
        if os.path.exists(audio_file):
            print(f'Creating transcript using OpenAIs Whisper for {audio_file}.')
            
            model = whisper.load_model("base")
            transcription = model.transcribe(audio_file)
            
            print('Exporting subtitle.')
            transcribe_audio(transcription, subtitle_file)
        else:
            print(f'Error. Audio file {audio_file} not created.')
        
    # Check if subtitle exists
    if os.path.exists(subtitle_file):
        print('Generating Audio Visual HTML Page')
        
        prj_name = os.path.basename(noext).replace(' ','_')
        prj_dir = os.path.join(out_dir, prj_name)
        if not(os.path.exists(prj_dir)):
            os.mkdir(prj_dir)

        line_list = file_to_line_list(subtitle_file)
        lines_indexed = chunk_sub_idx_to_list(line_list)

        html_list = create_html_list(prj_name, f, lines_indexed)

        html = create_html(prj_name, '\n'.join(html_list))

        save_path = os.path.join(prj_dir, prj_name + '.html')

        html_file = open(save_path, 'w', encoding='utf-8')
        html_file.write(html)
        html_file.close()
        
        print(f'Finished processing {f}')

    else:
        print(f'Error. Subtitle file {subtitle_file} not found.')

Generating Audio Visual HTML Page


chunk:   4%|██▌                                                        | 1/23 [2:06:04<46:13:49, 7564.96s/it, now=None]

MoviePy - Writing audio in Output\Naruto_HD_Set_1_Disc_1-001\Naruto_HD_Set_1_Disc_1-001_2000.mp3



chunk:   4%|██▌                                                        | 1/23 [2:06:05<46:13:53, 7565.14s/it, now=None]

MoviePy - Done.


...

chunk:   4%|██▌                                                        | 1/23 [2:16:07<49:54:39, 8167.24s/it, now=None]

MoviePy - Writing audio in Output\Naruto_HD_Set_1_Disc_1-001\Naruto_HD_Set_1_Disc_1-001_1410500.mp3



chunk:   4%|██▌                                                        | 1/23 [2:16:07<49:54:41, 8167.36s/it, now=None]

MoviePy - Done.
Finished processing Input\Naruto HD Set 1 Disc 1-001.mkv
Generating Audio Visual HTML Page


KeyError: 'video_fps'